In [1]:
import Foundation
import TensorFlow

In [2]:
let trainCSV = try String(contentsOfFile:"../data/train.csv", encoding: String.Encoding.utf8)
let testCSV = try String(contentsOfFile:"../data/test.csv", encoding: String.Encoding.utf8)

let trainRecords: [[Float]] = trainCSV.split(separator: "\n").map{ String($0).split(separator: ",").compactMap{ Float(String($0)) } }
let testRecords: [[Float]] = testCSV.split(separator: "\n").map{ String($0).split(separator: ",").compactMap{ Float(String($0)) } }

let numTrainRecords = trainRecords.count
let numTrainColumns = trainRecords[0].count
let numTestRecords = testRecords.count
let numTestColumns = testRecords[0].count

print(numTrainRecords, numTrainColumns, numTestRecords, numTestColumns)

let xTrain = trainRecords.map{ Array($0[0..<numTrainColumns-1]) }
let yTrain = trainRecords.map{ [$0[numTrainColumns-1]] }
let xTest = testRecords.map{ Array($0[0..<numTestColumns-1]) }
let yTest = testRecords.map{ [$0[numTestColumns-1]] }

print(xTrain.count, xTrain[0].count, yTrain.count, yTrain[0].count,
      xTest.count, xTest[0].count, yTest.count, yTest[0].count)

let xAllTrain = Array(xTrain.joined())
let yAllTrain = Array(yTrain.joined())
let xAllTest = Array(xTest.joined())
let yAllTest = Array(yTest.joined())

print(xAllTrain.count, yAllTrain.count, xAllTest.count, yAllTest.count)

405 14 101 14
405 13 405 1 101 13 101 1
5265 405 1313 101


In [3]:
let XTrain = Tensor<Float>(xAllTrain).reshaped(to: TensorShape([numTrainRecords, numTrainColumns-1]))
let YTrain = Tensor<Float>(yAllTrain).reshaped(to: TensorShape([numTrainRecords, 1]))
let XTest = Tensor<Float>(xAllTest).reshaped(to: TensorShape([numTestRecords, numTestColumns-1]))
let YTest = Tensor<Float>(yAllTest).reshaped(to: TensorShape([numTestRecords, 1]))

print(XTrain.shape, YTrain.shape, XTest.shape, YTest.shape)

[405, 13] [405, 1] [101, 13] [101, 1]


In [4]:
struct RegressionModel: Layer {
    var layer1 = Dense<Float>(inputSize: 13, outputSize: 64, activation: relu)
    var layer2 = Dense<Float>(inputSize: 64, outputSize: 32, activation: relu)
    var layer3 = Dense<Float>(inputSize: 32, outputSize: 1)
    
    @differentiable
    func callAsFunction(_ input: Tensor<Float>) -> Tensor<Float> {
        return input.sequenced(through: layer1, layer2, layer3)
    }
}

var model = RegressionModel()

In [5]:
let optimizer = RMSProp(for: model, learningRate: 0.001)
Context.local.learningPhase = .training

In [6]:
for _ in 0..<1000 {
    let 𝛁model = model.gradient { r -> Tensor<Float> in
        let ŷ = r(XTrain)
        let loss = meanSquaredError(predicted: ŷ, expected: YTrain)
        print("Loss: \(loss)")
        return loss
    }
    optimizer.update(&model, along: 𝛁model)
}

Loss: 598.97925
Loss: 589.64966
Loss: 583.1557
Loss: 577.6413
Loss: 572.6768
Loss: 568.08374
Loss: 563.75214
Loss: 559.5916
Loss: 555.56256
Loss: 551.614
Loss: 547.73157
Loss: 543.88666
Loss: 540.05084
Loss: 536.2087
Loss: 532.3809
Loss: 528.542
Loss: 524.6906
Loss: 520.7985
Loss: 516.87396
Loss: 512.92285
Loss: 508.94128
Loss: 504.91336
Loss: 500.83905
Loss: 496.71423
Loss: 492.52985
Loss: 488.2884
Loss: 483.97836
Loss: 479.61392
Loss: 475.2012
Loss: 470.74048
Loss: 466.21082
Loss: 461.62036
Loss: 456.96802
Loss: 452.26334
Loss: 447.50873
Loss: 442.69913
Loss: 437.83444
Loss: 432.91867
Loss: 427.95035
Loss: 422.92178
Loss: 417.84467
Loss: 412.71774
Loss: 407.53882
Loss: 402.3148
Loss: 397.04532
Loss: 391.72684
Loss: 386.36102
Loss: 380.95633
Loss: 375.51498
Loss: 370.03845
Loss: 364.52872
Loss: 358.994
Loss: 353.43283
Loss: 347.8434
Loss: 342.23413
Loss: 336.61008
Loss: 330.9728
Loss: 325.3237
Loss: 319.6651
Loss: 314.00082
Loss: 308.33313
Loss: 302.66708
Loss: 297.00143
Loss: 291.345

Loss: 6.278779
Loss: 6.270871
Loss: 6.264503
Loss: 6.2602334
Loss: 6.262123
Loss: 6.2715893
Loss: 6.286413
Loss: 6.2910457
Loss: 6.306396
Loss: 6.3151894
Loss: 6.2988935
Loss: 6.2495594
Loss: 6.217643
Loss: 6.192269
Loss: 6.175359
Loss: 6.1671133
Loss: 6.168162
Loss: 6.1723094
Loss: 6.177707
Loss: 6.1838694
Loss: 6.1637588
Loss: 6.141991
Loss: 6.1216555
Loss: 6.1092286
Loss: 6.0950527
Loss: 6.086775
Loss: 6.078113
Loss: 6.0761757
Loss: 6.0756145
Loss: 6.077569
Loss: 6.076438
Loss: 6.0805545
Loss: 6.0775948
Loss: 6.0964565
Loss: 6.131393
Loss: 6.111755
Loss: 6.0675564
Loss: 6.022523
Loss: 6.0064716
Loss: 6.0010386
Loss: 6.007497
Loss: 6.003655
Loss: 5.993161
Loss: 5.975675
Loss: 5.9640946
Loss: 5.94493
Loss: 5.931681
Loss: 5.918462
Loss: 5.914497
Loss: 5.9089327
Loss: 5.9121757
Loss: 5.9111032
Loss: 5.9195495
Loss: 5.9178085
Loss: 5.9227347
Loss: 5.9264355
Loss: 5.947149
Loss: 5.951105
Loss: 5.91375
Loss: 5.874357
Loss: 5.8396506
Loss: 5.8234386
Loss: 5.8137255
Loss: 5.815068
Loss: 5.82